In [1]:
from os import listdir
from time import time

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import math
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from utils import auc, load_dataset, plot_roc, train
from einops import rearrange

<IPython.core.display.Javascript object>

In [2]:
def schedule(epoch, lr) -> float:
    if epoch >= 200 and epoch % 25 == 0:
        lr = lr * math.exp(-0.1)
    return lr

<IPython.core.display.Javascript object>

In [4]:
scheduler = LearningRateScheduler(schedule)
es = EarlyStopping(monitor="loss", patience=5)
optimizer = Adam(lr=1e-4)

epochs = 1500
validation_freq = 2
# boundaries = [100, 300, 500, 1000, 2500, 5000]
fname = "model_2.h5"

<IPython.core.display.Javascript object>

In [5]:
model = load_model('../model_saves/evals/danq_small-2000.h5')

<IPython.core.display.Javascript object>

In [11]:
X_train, y_train, X_test, y_test = load_dataset("FOX")

X_train = rearrange(X_train, "w h c -> w c h")
X_test = rearrange(X_test, "w h c -> w c h")

y_train = rearrange(np.where(y_train < 5000, 0, 1), "w h -> h w")
y_test = rearrange(np.where(y_test < 5000, 0, 1), "w h -> h w")

<IPython.core.display.Javascript object>

In [37]:
for i, labels in enumerate(y_test):
    if np.all(labels == labels[0]):
        print("continue")
        continue
    print(i, auc(labels, model.predict(X_test)))
#     print(f"{i} accuracy: {model.evaluate(X_test, labels, verbose=0)[1]}")

0 0.4815097940979465
1 0.4775075075075076
2 0.4864659436747327
3 0.432095676541233
4 0.4854589187120757
5 0.5315396697510518
6 0.6109331197436925
7 0.560958395989975
8 0.4652246219097048
9 0.5502682606469422
10 0.43753058298196645
11 0.5039979899497486
12 0.490185370741483
13 0.47394890364111847
14 0.4632103982744783
15 0.399459513562206
16 0.4969922889991231
17 0.5748702632171466
18 0.5545291737087745
19 0.5431595276220976
20 0.530642878586439
continue
22 0.506256986788618
23 0.5261726547209307
24 0.5268678874459789
25 0.4312197049870001
26 0.5305691871140044
27 0.5005227546659848
28 0.5338293169090307
29 0.4679502965847904
continue


<IPython.core.display.Javascript object>